In [17]:
import math
import pickle
import random
from typing import List, Tuple
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import numpy as np
import torch
from catboost.datasets import msrank_10k
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from tqdm.auto import tqdm
from sklearn.model_selection import cross_val_score

class Solution:
    def __init__(self, n_estimators: int = 100, lr: float = 0.5, ndcg_top_k: int = 10,
                 subsample: float = 0.9, colsample_bytree: float = 0.9,
                 max_depth: int = 5, min_samples_leaf: int = 8):
        self._prepare_data()

        self.ndcg_top_k = ndcg_top_k
        self.n_estimators = n_estimators
        self.lr = lr
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.trees = []
        self.indices = []


    def _get_data(self) -> List[np.ndarray]:
        train_df, test_df = msrank_10k()

        X_train = train_df.drop([0, 1], axis=1).values
        y_train = train_df[0].values
        query_ids_train = train_df[1].values.astype(int)

        X_test = test_df.drop([0, 1], axis=1).values
        y_test = test_df[0].values
        query_ids_test = test_df[1].values.astype(int)

        return [X_train, y_train, query_ids_train, X_test, y_test, query_ids_test]

    def _prepare_data(self) -> None:
        (X_train, y_train, self.query_ids_train,
            X_test, y_test, self.query_ids_test) = self._get_data()
        
        X_train = self._scale_features_in_query_groups(X_train,self.query_ids_train)
        X_test = self._scale_features_in_query_groups(X_test,self.query_ids_test)
        self.X_train = torch.FloatTensor(X_train)
        self.X_test = torch.FloatTensor(X_test)
        self.ys_train = torch.reshape(torch.FloatTensor(y_train),(len(y_train), 1))
        self.ys_test = torch.reshape(torch.FloatTensor(y_test),(len(y_test), 1))
        pass

    def _scale_features_in_query_groups(self, inp_feat_array: np.ndarray,
                                        inp_query_ids: np.ndarray) -> np.ndarray:
        scaler = StandardScaler()
        
        y_ids = inp_query_ids
        inp_f = inp_feat_array
        
        y_uniques = np.unique(y_ids)
        
        for i,yi in enumerate(y_uniques):
            scaler = StandardScaler()
            inp_i = scaler.fit_transform(inp_f[y_ids == yi])

            if i == 0:
                
                inp_all = inp_i
            else:
                inp_all = np.append(inp_all,inp_i,axis = 0)  

        return inp_all

    def _train_one_tree(self, cur_tree_idx: int,
                        train_preds: torch.FloatTensor
                        ,param) -> Tuple[DecisionTreeRegressor, np.ndarray]:
        np.random.seed(cur_tree_idx)
        new_lambdas = torch.zeros(train_preds.size())
        for i , cur_id in enumerate(np.unique(self.query_ids_train)) :
            mask_train = self.query_ids_train == cur_id
            new_lambdas[mask_train] = self._compute_lambdas(self.ys_train[mask_train] ,
                                                            train_preds[mask_train])

        idx_col = np.random.choice(self.X_train.size(1) , int(self.X_train.size(1) * self.colsample_bytree))
        idx_sub = np.random.choice(train_preds.size(0) , int(train_preds.size(0) * self.subsample))
        parameter = { "splitter" : "best" ,
                      "max_depth" : self.max_depth ,
                      "min_samples_leaf" : self.min_samples_leaf ,
                      "min_weight_fraction_leaf" : 0.4 ,
                      "max_leaf_nodes" : 9 }
        
        model = DecisionTreeRegressor(**param)
        model.fit(self.X_train[idx_sub][: , idx_col] , torch.reshape(torch.flatten(new_lambdas)[idx_sub] , (-1 , 1)))
        return model , idx_col

    def _calc_data_ndcg(self, queries_list: np.ndarray,
                        true_labels: torch.FloatTensor, preds: torch.FloatTensor) -> float:
        ndcg_s = []
        for i,cur_id in enumerate(np.unique(queries_list)):
            mask_train = queries_list == cur_id
            cur_ndcg = self._ndcg_k(true_labels[mask_train],
                                                       preds[mask_train],self.ndcg_top_k) 
            if np.isnan(cur_ndcg):
                ndcg_s.append(0)
                continue
                
            ndcg_s.append(cur_ndcg) 
        return np.mean(ndcg_s)

    def fit(self):
        np.random.seed(0)

        space4dt = {

            'max_depth': hp.choice('max_depth', range(2,90)),
            "max_leaf_nodes":hp.choice("max_leaf_nodes",range(10,90)),
            "min_samples_split":hp.choice("min_samples_split",range(10,90)),
            "min_samples_leaf":hp.choice("min_samples_leaf",range(10,90)), 
            "min_weight_fraction_leaf" : hp.choice("min_weight_fraction_leaf",[0.012]),
            "random_state" : hp.choice("random_state",[14,20,150])                      
        }    
        def f(params):     
            y_pred_train = torch.zeros(self.ys_train.size())
            y_pred_test = torch.zeros(self.ys_test.size())
            models = []
            indices_all = []

            ndcg_best = 0
            best_tree = 0
                
            for tree in range(self.n_estimators):
                model, indices = self._train_one_tree(tree , y_pred_train, params)
                models.append(model)
                indices_all.append(indices)
                self.trees = models
                self.indices = indices_all

                y_pred_temp_train = torch.Tensor(model.predict(self.X_train[: , indices]))
                y_pred_temp_test = torch.Tensor(model.predict(self.X_test[: , indices]))

                y_pred_temp_train = torch.reshape(y_pred_temp_train , y_pred_train.size())
                y_pred_temp_test = torch.reshape(y_pred_temp_test , y_pred_test.size())

                y_pred_train -= self.lr * y_pred_temp_train 
                y_pred_test -= self.lr * y_pred_temp_test 

                ndcg_test = self._calc_data_ndcg(self.query_ids_test ,
                                                torch.flatten(self.ys_test) , torch.flatten(y_pred_test))
                ndcg_train = self._calc_data_ndcg(self.query_ids_train ,
                                                torch.flatten(self.ys_train) , torch.flatten(y_pred_train))
#                 print(f"размер выборок test:{y_pred_test.size()}  train: {y_pred_train.size()}")
                preds_test = self.predict(self.X_test)
#                 print('Test NDCG из predict:', self._calc_data_ndcg(self.query_ids_test, 
#                                                          torch.flatten(self.ys_test), torch.flatten(preds_test)))
                if tree % 50 == 0: 
                    print(f"tree_num:{tree}, test: {ndcg_test}, train:{ndcg_train}" )
                if ndcg_test > ndcg_best :
                    ndcg_best = ndcg_test
                    best_tree = tree
                if tree == 99:
                    print(model.get_params())
                
            return {'loss': 1 - ndcg_best, 'status': STATUS_OK} 
        
        trials = Trials()
        best = fmin(f, space4dt, algo=tpe.suggest, max_evals=15, trials=trials)
        print ('best:', best)
        print(best)
        self.trials = trials
        
        
        # Best model
        y_pred_train = torch.zeros(self.ys_train.size())
        y_pred_test = torch.zeros(self.ys_test.size())
        models = []
        indices_all = []

        ndcg_best = 0
        best_tree = 0

        for tree in range(self.n_estimators) :

            model, indices = self._train_one_tree(tree , y_pred_train,best)

            models.append(model)
            indices_all.append(indices)
            self.trees = models
            self.indices = indices_all

            y_pred_temp_train = torch.Tensor(model.predict(self.X_train[: , indices]))
            y_pred_temp_test = torch.Tensor(model.predict(self.X_test[: , indices]))

            y_pred_temp_train = torch.reshape(y_pred_temp_train , y_pred_train.size())
            y_pred_temp_test = torch.reshape(y_pred_temp_test , y_pred_test.size())
            
            y_pred_train -= self.lr * y_pred_temp_train 
            y_pred_test -= self.lr * y_pred_temp_test 
            
            ndcg_test = self._calc_data_ndcg(self.query_ids_test ,
                                            torch.flatten(self.ys_test) , torch.flatten(y_pred_test))
            ndcg_train = self._calc_data_ndcg(self.query_ids_train ,
                                            torch.flatten(self.ys_train) , torch.flatten(y_pred_train))

            preds_test = self.predict(self.X_test)
#             print('Test NDCG из predict:', self._calc_data_ndcg(self.query_ids_test, 
#                                                      torch.flatten(self.ys_test), torch.flatten(preds_test)))
            if tree % 50 == 0:             
                print(f"tree number:{tree}, test: {ndcg_test}, train:{ndcg_train}" )
            if ndcg_test > ndcg_best :
                ndcg_best = ndcg_test
                best_tree = tree

        self.indices = indices_all[:best_tree+1] 
        self.best_ndcg = ndcg_best
        self.trees = models[:best_tree+1]
        
        print(ndcg_best,best_tree)
        
        self.save_model("model.sav")
        pass

    def predict(self, data: torch.FloatTensor) -> torch.FloatTensor:
        
#         self.load_model( "model.sav")
        
        y_pred_temp = torch.flatten(torch.zeros(self.ys_train.size())) # first forecast 
        for i in range(len(self.trees)):

            tree_pred = torch.flatten(torch.Tensor(self.trees[i].predict(data[:,self.indices[i]]))) # tree forecast  
            y_pred_temp -=self.lr*tree_pred # grad boost step 
            
        return y_pred_temp  

    def _compute_lambdas(self, y_true: torch.FloatTensor, y_pred: torch.FloatTensor) -> torch.FloatTensor:
        def compute_labels_in_batch(y_true):

            # the difference of relevance of each with each object
            rel_diff = y_true - y_true.t()

            # 1 if more relevant
            pos_pairs = (rel_diff > 0).type(torch.float32)

            # 1 if less relevant
            neg_pairs = (rel_diff < 0).type(torch.float32)
            Sij = pos_pairs - neg_pairs
            return Sij

        def compute_gain_diff(y_true, gain_scheme):
            if gain_scheme == "exp2":
                gain_diff = torch.pow(2.0, y_true) - torch.pow(2.0, y_true.t())
            elif gain_scheme == "diff":
                gain_diff = y_true - y_true.t()
            else:
                raise ValueError(f"{gain_scheme} method not supported")
            return gain_diff

        def compute_gain(y_value: float, gain_scheme: str) -> float:
            if gain_scheme == "const":
                gain = y_value

            if gain_scheme == "exp2":   
                gain = 2**y_value-1

            return gain
        
        def compute_ideal_dcg(ys_true: torch.Tensor, gain_scheme: str) -> float:
            values_true , indices_true = torch.flatten(ys_true).sort(descending = True)
            gain_id = 0
            for i in range(len(values_true)):   
                gain_id+= float(compute_gain(values_true[i].item(),gain_scheme)/math.log2(i+2))
            return float(gain_id)
        
        ideal_dcg = compute_ideal_dcg(torch.flatten(y_true), gain_scheme="exp2")

        if ideal_dcg == 0:
            N = 0
        else: 

            N = 1 / ideal_dcg

        # calculate the order of documents according to relevance estimates
        _, rank_order = torch.sort(y_true, descending=True, axis=0)
        rank_order += 1
        ndcg_scheme= "exp2"
        with torch.no_grad():
            # we get all pairwise pressure differences in the butch
            pos_pairs_score_diff = 1.0 + torch.exp((y_pred - y_pred.t()))

            # we put the markup for pairs, 1 if the first document is more relevant
            # -1 if the second document is more relevant
            Sij = compute_labels_in_batch(y_true)
            # calculate the gain change due to permutations
            gain_diff = compute_gain_diff(y_true, ndcg_scheme)

            # calculate the change in the denominators-discounters
            decay_diff = (1.0 / torch.log2(rank_order + 1.0)) - (1.0 / torch.log2(rank_order.t() + 1.0))
            # calculate nDCG bies
            delta_ndcg = torch.abs(N * gain_diff * decay_diff)
            # calculate lambdas
            lambda_update =  (0.5 * (1 - Sij) - 1 / pos_pairs_score_diff) * delta_ndcg
            lambda_update = torch.sum(lambda_update, dim=1, keepdim=True)

            return lambda_update


    def _ndcg_k(self , ys_true , ys_pred , ndcg_top_k) -> float :

        def compute_gain(y_value: float , gain_scheme: str) -> float :
            if gain_scheme == "const" :
                gain = y_value

            if gain_scheme == "exp2" :
                gain = 2 ** y_value - 1
            return gain

        def dcg(ys_true: torch.Tensor , ys_pred: torch.Tensor , gain_scheme: str , ndcg_top_k) -> float :
            values_pred , indices_pred = ys_pred.sort(descending=True)
            ys_true = ys_true[indices_pred]
            gain = 0
            for i in range(len(ys_true[:ndcg_top_k])) :
                gain += float(compute_gain(ys_true[i].item() , gain_scheme) / math.log2(i + 2))
            return float(gain)

        gain = dcg(ys_true , ys_pred , "exp2" , ndcg_top_k)

        gain_ideal = dcg(ys_true , ys_true , "exp2" , ndcg_top_k)

        if gain_ideal == 0:
            ndcg = 0
        else:
            ndcg = gain / gain_ideal
        return ndcg     
    
    def save_model(self, path: str):
        state = {"trees":self.trees,
                "indices":self.indices,
                "lr":self.lr}
        f = open(path, 'wb')
        pickle.dump(state, f)
        pass

            
    def load_model(self, path: str):
        all_model = pickle.load(open(path , 'rb'))
        self.trees , self.indices , self.lr = all_model["trees"] , all_model["indices"] , all_model["lr"]
        pass


In [18]:
sol = Solution()
sol.fit()
# best_params = {'max_depth': 7, 'max_leaf_nodes': 17, 'min_samples_leaf': 16, 'min_samples_split': 11, 'min_weight_fraction_leaf': 0.012}

tree_num:0, test: 0.37045285175485515, train:0.4112899819120048                 
tree_num:50, test: 0.40517524409384703, train:0.6792054999217738                
{'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 55, 'max_features': None, 'max_leaf_nodes': 51, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 15, 'min_samples_split': 84, 'min_weight_fraction_leaf': 0.012, 'random_state': 20, 'splitter': 'best'}
tree_num:0, test: 0.37489140158732664, train:0.4126823294928434                 
tree_num:50, test: 0.41655861587823767, train:0.6916886796083347                
{'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 26, 'max_features': None, 'max_leaf_nodes': 85, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 80, 'min_samples_split': 34, 'min_weight_fraction_leaf': 0.012, 'random_state': 150, 'splitter': 'best'}
tree_num:0, test: 0.37489140158732664, train:0.4126823294928434                 
tree_num:50, test: 0.41655861587823767, train:0.6916886796083347  

In [308]:
sol.trees[0].get_params()

{'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': 21,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 19,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0,
 'random_state': None,
 'splitter': 'best'}